In [ ]:
import matplotlib.pyplot as pp
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tp

from functools import partial

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
pp.style.use('ggplot')

# Coin flip

In [ ]:
trials = [0, 1, 2, 3, 4, 5, 8, 15, 50, 500, 1000, 2000]
data = tp.distributions.Bernoulli(probs=0.5).sample(trials[-1])
sums = [tf.reduce_sum(data[:trial]) for trial in trials]

prior_alpha = 1
prior_beta = 1

points = tf.linspace(start=0.0, stop=1.0, num=1000, name='linspace')
posterior_densities = [
    tp.distributions.Beta(concentration1=tf.to_float(prior_alpha + sum),
                          concentration0=tf.to_float(prior_beta + trial - sum))
    .prob(points)
    for trial, sum in zip(trials, sums)
]

In [ ]:
sums_stacked = tf.stack(sums)
posterior_densities_stacked = tf.stack(posterior_densities)

session = tf.Session()

[
    data_,
    sums_stacked_,
    points_,
    posterior_densities_stacked_,
] = session.run([
    data,
    sums_stacked,
    points,
    posterior_densities_stacked,
])

In [ ]:
pp.figure(figsize=(10, 12))
for i in range(len(trials)):
    axis = pp.subplot(len(trials) / 2, 2, i + 1)
    pp.setp(axis.get_yticklabels(), visible=False)
    pp.plot(points_, posterior_densities_stacked_[i], 
            label='%d tosses with %d heads' % (trials[i], sums_stacked_[i]))
    pp.fill_between(points_, 0, posterior_densities_stacked_[i], alpha=0.4)
    pp.axvline(x=0.5, color='black', linestyle='--', lw=1)
    pp.legend().get_frame().set_alpha(0.4)
    pp.autoscale(tight=True)
pp.tight_layout()

# Text messages

In [ ]:
tf.reset_default_graph()

In [ ]:
data = [
    13, 24, 8,  24,  7, 35, 14, 11, 15, 11, 22, 22, 11, 57,
    11, 19, 29,  6, 19, 12, 22, 12, 18, 72, 32,  9,  7, 13,
    19, 23, 27, 20,  6, 17, 13, 10, 14,  6, 16, 15,  7,  2,
    15, 15, 19, 70, 49,  7, 53, 22, 21, 31, 19, 11, 18, 20,
    12, 35, 17, 23, 17,  4,  2, 31, 30, 13, 27,  0, 39, 37,
    5,  14, 13, 22,
]
observation_num = len(data)
data = tf.constant(data, dtype=tf.float32)

In [ ]:
def unnormalized_log_probability(data, lambda_1, lambda_2, tau):
    alpha = 1.0 / tf.reduce_mean(data)
    rv_lambda_1 = tp.distributions.Exponential(rate=alpha)
    rv_lambda_2 = tp.distributions.Exponential(rate=alpha)
    rv_tau = tp.distributions.Uniform()
    indices = tf.range(tf.size(data))
    indices = tf.to_int32(tau * tf.to_float(tf.size(data)) <= tf.to_float(indices))
    lambda_12 = tf.gather([lambda_1, lambda_2], indices=indices)
    rv_observation = tp.distributions.Poisson(rate=lambda_12)
    return (
        rv_lambda_1.log_prob(lambda_1) +
        rv_lambda_2.log_prob(lambda_2) +
        rv_tau.log_prob(tau) +
        tf.reduce_sum(rv_observation.log_prob(data))
    )

state = [
    tf.reduce_mean(data),
    tf.reduce_mean(data),
    0.5,
]

bijector = [
    tp.bijectors.Exp(),
    tp.bijectors.Exp(),
    tp.bijectors.Sigmoid(), 
]

with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    step_size = tf.get_variable(name='step_size',
                                initializer=tf.constant(0.05),
                                trainable=False,
                                use_resource=True)

kernel = tp.mcmc.TransformedTransitionKernel(
    inner_kernel=tp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=partial(unnormalized_log_probability, data),
        num_leapfrog_steps=2,
        step_size=step_size,
        step_size_update_fn=tp.mcmc.make_simple_step_size_update_policy(),
        state_gradients_are_stopped=True,
    ),
    bijector=bijector,
)

[lambda_1, lambda_2, tau], kernel = tp.mcmc.sample_chain(
    num_results=100000,
    num_burnin_steps=10000,
    current_state=state,
    kernel=kernel,
)
tau = tf.floor(tau * tf.to_float(tf.size(data)))

In [ ]:
session = tf.Session()

session.run([
    tf.global_variables_initializer(),
    tf.local_variables_initializer(),
])

[
    lambda_1_,
    lambda_2_,
    tau_,
    kernel_,
] = session.run([
    lambda_1,
    lambda_2,
    tau,
    kernel,
])
    
print('Acceptance rate: {}'.format(kernel_.inner_results.is_accepted.mean()))
print('Final step size: {}'.format(kernel_.inner_results.extra.step_size_assign[-100:].mean()))

In [ ]:
pp.figure(figsize=(10, 4))
pp.hist(tau_, bins=observation_num, density=True)
pp.xticks(np.arange(observation_num))
pp.xlim([37, 47])
pp.xlabel('Day')
pp.ylabel('Posterior density');